# Train a transformer to write Tang poem

Refer to https://pytorch.org/hub/huggingface_pytorch-transformers/ (torch tutorial)

https://github.com/ckiplab/ckip-transformers (Pretrained Chinese Model)

https://github.com/huggingface/transformers/tree/master/examples/pytorch/language-modeling (examples)

In [299]:
from itertools import chain
import pickle
import math
import numpy as np
import re

import torch
import torch.nn
from torch.utils.data import DataLoader
import transformers
from transformers import (
    AdamW,
    CONFIG_MAPPING,
    MODEL_FOR_CAUSAL_LM_MAPPING,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
    get_scheduler,
    BertTokenizerFast
)

from dataclasses import dataclass, field
import datasets
from datasets import load_dataset
from typing import Optional

In [ ]:
model.load_state_dict

# 0) Parse arguments

In [179]:
MODEL_CONFIG_CLASSES = list(MODEL_FOR_CAUSAL_LM_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The model checkpoint for weights initialization."
            "Don't set if you want to train a model from scratch."
        },
    )
    model_type: Optional[str] = field(
        default=None,
        metadata={"help": "If training from scratch, pass a model type from the list: " + ", ".join(MODEL_TYPES)},
    )
    config_overrides: Optional[str] = field(
        default=None,
        metadata={
            "help": "Override some existing default config settings when a model is trained from scratch. Example: "
            "n_embd=10,resid_pdrop=0.2,scale_attn_weights=false,summary_type=cls_index"
        },
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )

    def __post_init__(self):
        if self.config_overrides is not None and (self.config_name is not None or self.model_name_or_path is not None):
            raise ValueError(
                "--config_overrides can't be used in combination with --config_name or --model_name_or_path"
            )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default='QuanTangPoem', metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a text file)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."},
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )

    block_size: Optional[int] = field(
        default=None,
        metadata={
            "help": "Optional input sequence length after tokenization. "
            "The training dataset will be truncated in block of this size for training. "
            "Default to the model max input length for single sentence inputs (take into account special tokens)."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    validation_split_percentage: Optional[int] = field(
        default=5,
        metadata={
            "help": "The percentage of the train set used as validation set in case there's no validation split"
        },
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    keep_linebreaks: bool = field(
        default=True, metadata={"help": "Whether to keep line breaks when using TXT files or not."}
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json", "txt"], "`train_file` should be a csv, a json or a txt file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json", "txt"], "`validation_file` should be a csv, a json or a txt file."

In [181]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses(look_for_args_file=False, args=[
        '--output_dir', './output/2_transformer_torch',
        '--warmup_steps', '500',
        '--learning_rate', '0.00003',
        '--weight_decay', '0.01',
        '--adam_epsilon', '1e-6',
        '--max_steps', '3000',
        '--logging_steps', '500',
        '--save_steps', '500',
        '--max_grad_norm', '5.0',
        '--per_device_eval_batch_size', '2',
        '--per_device_train_batch_size', '2',
        '--gradient_accumulation_steps', '4',
        '--do_train',
        '--do_eval',
        '--fp16',
        '--fp16_opt_level', 'O2',
    ])

# 1) Load model

In [496]:
# Load a pre-trained CasualLM for Chinese
model = AutoModelForCausalLM.from_pretrained('ckiplab/gpt2-base-chinese')

loading configuration file https://huggingface.co/ckiplab/gpt2-base-chinese/resolve/main/config.json from cache at /home2/swan15/.cache/huggingface/transformers/6145f6ee276deb91b321aa10f067156bd20746022f0de671ffc8f63d88659d5d.2f94726577a800caccb98e111070d99b55b8bd9377c3c27f439f11eabcb77a61
Model config GPT2Config {
  "_name_or_path": "ckiplab/gpt2-base-chinese",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 101,
  "embd_pdrop": 0.1,
  "eos_token_id": 102,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "sum

In [17]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [34]:
model.config

GPT2Config {
  "_name_or_path": "ckiplab/gpt2-base-chinese",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 101,
  "embd_pdrop": 0.1,
  "eos_token_id": 102,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "tokenizer_class": "BertTokenizerFast",
  "transformers_version": "4.15.0",
  "use_cache": true,
  "vocab_size": 21128
}

# 2) Training 

* public datasets available at https://huggingface.co/datasets/

* loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at https://huggingface.co/docs/datasets/loading_datasets.html.

In [66]:
# Configs
train_file = "../data/全唐诗_processed.txt"
cache_dir = "../data/cached"
validation = True

In [40]:
set_seed(123)

## 2.1) Constructing training data

In [256]:
filename = "../data/全唐诗.txt"

def add_lines(old_line, line, poems):
    #print line
    if len(line) < 12 or re.search(u'\u3010', line) is not None:
        '''If this line is blank or title'''
        old_line = ''
    elif old_line == '':
        '''If start a new line'''
        poems.append(line[0:-2])
        old_line = line
    else:
        '''If continuing to last poem'''
        poems[-1] = poems[-1]+line[0:-2]
        old_line = line
    return poems, old_line

print("Reading txt file...")
poems = []
old_line = ''
with open(filename, 'rb') as f:
    while True:
        try:
            line = f.readline().decode("cp936")
        except:
            continue
            
        if not line:
            break
            
        poems, old_line = add_lines(old_line, line, poems)
        
print("Writing processed.txt...")
with open(train_file, 'w') as f:
    for _ in poems:
        f.writelines(_)
        f.writelines("\n")


Reading txt file...
Writing processed.txt...


## 2.2) Create processed.txt

In [263]:
extension = train_file.split(".")[-1]
dataset_args = dict()
if extension == "txt":
    extension = "text"
    dataset_args["keep_linebreaks"] = True
data_files = {}
data_files["train"] = train_file
raw_datasets = load_dataset(extension, data_files=data_files, cache_dir=cache_dir)
# If no validation data is there, validation_split_percentage will be used to divide the dataset.

if validation:
    raw_datasets["validation"] = load_dataset(
        extension,
        data_files=data_files,
        split=f"train[:{1}%]",
        cache_dir=cache_dir,
        **dataset_args,
    )
    raw_datasets["train"] = load_dataset(
        extension,
        data_files=data_files,
        split=f"train[{1}%:]",
        cache_dir=cache_dir,
        **dataset_args,
    )

W0119 03:42:45.354353 46912496430720 builder.py:379] Using custom data configuration default-ab9e24fddc2c23d2
W0119 03:42:45.361300 46912496430720 builder.py:532] Reusing dataset text (../data/cached/text/default-ab9e24fddc2c23d2/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4)


  0%|          | 0/1 [00:00<?, ?it/s]

W0119 03:42:45.585376 46912496430720 builder.py:379] Using custom data configuration default-86cd11d20e1ce057
W0119 03:42:45.590573 46912496430720 builder.py:532] Reusing dataset text (../data/cached/text/default-86cd11d20e1ce057/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4)
W0119 03:42:45.794228 46912496430720 builder.py:379] Using custom data configuration default-86cd11d20e1ce057
W0119 03:42:45.799307 46912496430720 builder.py:532] Reusing dataset text (../data/cached/text/default-86cd11d20e1ce057/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4)


In [403]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model.resize_token_embeddings(len(tokenizer))

https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /home2/swan15/.cache/huggingface/transformers/tmpabnr6cc6


Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt in cache at /home2/swan15/.cache/huggingface/transformers/36acdf4f3edf0a14ffb2b2c68ba47e93abd9448825202377ddb16dae8114fe07.accd894ff58c6ff7bd4f3072890776c14f4ea34fcc08e79cd88c2d157756dceb
creating metadata file for /home2/swan15/.cache/huggingface/transformers/36acdf4f3edf0a14ffb2b2c68ba47e93abd9448825202377ddb16dae8114fe07.accd894ff58c6ff7bd4f3072890776c14f4ea34fcc08e79cd88c2d157756dceb
https://huggingface.co/bert-base-chinese/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /home2/swan15/.cache/huggingface/transformers/tmp85jvyukn


Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-chinese/resolve/main/tokenizer.json in cache at /home2/swan15/.cache/huggingface/transformers/7e23f4e1f58f867d672f84d9a459826e41cea3be6d0fe62502ddce9920f57e48.4495f7812b44ff0568ce7c4ff3fdbb2bac5eaf330440ffa30f46893bf749184d
creating metadata file for /home2/swan15/.cache/huggingface/transformers/7e23f4e1f58f867d672f84d9a459826e41cea3be6d0fe62502ddce9920f57e48.4495f7812b44ff0568ce7c4ff3fdbb2bac5eaf330440ffa30f46893bf749184d
https://huggingface.co/bert-base-chinese/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /home2/swan15/.cache/huggingface/transformers/tmp296404tk


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-chinese/resolve/main/tokenizer_config.json in cache at /home2/swan15/.cache/huggingface/transformers/2dc6085404c55008ba7fc09ab7483ef3f0a4ca2496ccee0cdbf51c2b5d529dff.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
creating metadata file for /home2/swan15/.cache/huggingface/transformers/2dc6085404c55008ba7fc09ab7483ef3f0a4ca2496ccee0cdbf51c2b5d529dff.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt from cache at /home2/swan15/.cache/huggingface/transformers/36acdf4f3edf0a14ffb2b2c68ba47e93abd9448825202377ddb16dae8114fe07.accd894ff58c6ff7bd4f3072890776c14f4ea34fcc08e79cd88c2d157756dceb
loading file https://huggingface.co/bert-base-chinese/resolve/main/tokenizer.json from cache at /home2/swan15/.cache/huggingface/transformers/7e23f4e1f58f867d672f84d9a459826e41cea3be6d0fe62502ddce9920f57e48.4495f7812b44ff0568ce7c4ff3fdbb2bac5eaf330440ffa30f4

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-chinese/resolve/main/config.json in cache at /home2/swan15/.cache/huggingface/transformers/6cc404ca8136bc87bae0fb24f2259904943d776a6c5ddc26598bbdc319476f42.0f9bcd8314d841c06633e7b92b04509f1802c16796ee67b0f1177065739e24ae
creating metadata file for /home2/swan15/.cache/huggingface/transformers/6cc404ca8136bc87bae0fb24f2259904943d776a6c5ddc26598bbdc319476f42.0f9bcd8314d841c06633e7b92b04509f1802c16796ee67b0f1177065739e24ae
loading configuration file https://huggingface.co/bert-base-chinese/resolve/main/config.json from cache at /home2/swan15/.cache/huggingface/transformers/6cc404ca8136bc87bae0fb24f2259904943d776a6c5ddc26598bbdc319476f42.0f9bcd8314d841c06633e7b92b04509f1802c16796ee67b0f1177065739e24ae
Model config BertConfig {
  "_name_or_path": "bert-base-chinese",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout

Embedding(21128, 768)

In [275]:
def tokenize_function(examples):
    output = tokenizer(examples['text'])
    return output

tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    num_proc=2, # num_proc=data_args.preprocessing_num_workers,
    # remove_columns=column_names,
    # load_from_cache_file=not data_args.overwrite_cache,
    desc="Running tokenizer on dataset",
)

W0119 03:45:52.539324 46912496430720 arrow_dataset.py:2310] Loading cached processed dataset at ../data/cached/text/default-86cd11d20e1ce057/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4/cache-85050f9ff5597adb.arrow
W0119 03:45:52.689638 46912496430720 arrow_dataset.py:2310] Loading cached processed dataset at ../data/cached/text/default-86cd11d20e1ce057/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4/cache-084357e983100896.arrow
W0119 03:45:53.114873 46912496430720 arrow_dataset.py:2310] Loading cached processed dataset at ../data/cached/text/default-86cd11d20e1ce057/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4/cache-0ab826c945b682f1.arrow
W0119 03:45:53.114873 46912496430720 arrow_dataset.py:2310] Loading cached processed dataset at ../data/cached/text/default-86cd11d20e1ce057/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4/cache-4889eeca32214c2e.arrow


In [269]:
block_size = tokenizer.model_max_length

In [320]:
# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):    
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
        # if k != 'text'
    }
    result["labels"] = result["input_ids"].copy()
    
    '''for default batch loader'''
    # if 'text' in list(result.keys()):
    #     result.pop('text')
    
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=2, # num_proc=data_args.preprocessing_num_workers,
    # load_from_cache_file=not data_args.overwrite_cache,
    desc=f"Grouping texts in chunks of {block_size}",
)

In [271]:
train_dataset = lm_datasets["train"]
eval_dataset = lm_datasets["validation"]

In [414]:
from transformers import BatchEncoding
device = "cuda"
def my_default_data_collator(features, device=device):
    import torch

    if not isinstance(features[0], (dict, BatchEncoding)):
        features = [vars(f) for f in features]
    first = features[0]
    batch = {}

    # Special handling for labels.
    # Ensure that tensor is created with the correct type
    # (it should be automatically the case, but let's make sure of it.)
    if "label" in first and first["label"] is not None:
        label = first["label"].item() if isinstance(first["label"], torch.Tensor) else first["label"]
        dtype = torch.long if isinstance(label, int) else torch.float
        batch["labels"] = torch.tensor([f["label"] for f in features], dtype=dtype)
    elif "label_ids" in first and first["label_ids"] is not None:
        if isinstance(first["label_ids"], torch.Tensor):
            batch["labels"] = torch.stack([f["label_ids"] for f in features])
        else:
            dtype = torch.long if type(first["label_ids"][0]) is int else torch.float
            batch["labels"] = torch.tensor([f["label_ids"] for f in features], dtype=dtype)

    # Handling of all other possible keys.
    # Again, we will use the first element to figure out which key/values are not None for this model.
    for k, v in first.items():
        if k not in ("label", "label_ids", "text") and v is not None and not isinstance(v, str):
            if isinstance(v, torch.Tensor):
                batch[k] = torch.stack([f[k] for f in features])
            else:
                batch[k] = torch.tensor([f[k] for f in features])
                
    for k in list(batch.keys()):
        batch[k] = batch[k].to(device)

    return batch

In [415]:
# DataLoaders creation:
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=my_default_data_collator, batch_size=training_args.per_device_train_batch_size
)
eval_dataloader = DataLoader(
    eval_dataset, collate_fn=my_default_data_collator, batch_size=training_args.per_device_eval_batch_size
)

In [502]:
training_args.max_steps = 500000 # was 3000
training_args.num_train_epochs = 500 # was 5
training_args.warmup_steps = 0 # was 500
training_args.learning_rate = 3e-5 # was 3e-5

In [503]:
# Optimizer
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": training_args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=training_args.learning_rate)

In [504]:
# Scheduler
lr_scheduler = get_scheduler(
    name=training_args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=training_args.warmup_steps,
    num_training_steps=training_args.max_steps)

In [ ]:
# Train!
completed_steps = 0
model.cuda()
for epoch in range(training_args.num_train_epochs):
    model.train()
    losses = []
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / training_args.gradient_accumulation_steps
        # accelerator.backward(loss)
        loss.backward()
        if step % training_args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1

        if completed_steps >= training_args.max_steps:
            break
            
        losses.append(loss.item())
    print("Train loss: {}".format(np.mean(losses)))
            

    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(loss.item())

    losses = torch.tensor(losses[:len(eval_dataset)])
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f"epoch {epoch}: perplexity: {perplexity} loss: {torch.mean(losses)}")

    # Try some outputs
    def sample(preds, temperature=1.1):
        # helper function to sample an index from a probability array
        preds = np.asarray(preds).astype('float64')
        # preds = np.log(preds) / temperature
        preds /= temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)
    
    model.eval()
    text = '明月几时'
    for i in range(100):
        indexed_token = tokenizer.encode(text)
        tokens_tensor = torch.tensor(indexed_token).to('cuda')
        with torch.no_grad():
            outputs = model(tokens_tensor)

        predictions = outputs[0]
        # predicted_index = torch.argmax(predictions[-1, :]).item()
        predicted_index = sample(predictions[-2, :].cpu().numpy())
        predicted_text = tokenizer.decode([predicted_index])
        text += predicted_text
    print(text)

In [516]:
if training_args.output_dir is not None:
    model.save_pretrained(training_args.output_dir)
    tokenizer.save_pretrained(training_args.output_dir)

Configuration saved in ./output/2_transformer_torch/config.json
Model weights saved in ./output/2_transformer_torch/pytorch_model.bin
tokenizer config file saved in ./output/2_transformer_torch/tokenizer_config.json
Special tokens file saved in ./output/2_transformer_torch/special_tokens_map.json


# Writing! 

In [525]:
model.eval()
text = '明月几时有'
for i in range(400):
    indexed_token = tokenizer.encode(text)
    tokens_tensor = torch.tensor(indexed_token).to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)

    predictions = outputs[0]
    predicted_index = sample(predictions[-2, :].cpu().numpy())
    predicted_text = tokenizer.decode([predicted_index])
    text += predicted_text
    if len(text) > 512:
        text = text[-512:]
print(text)

明月几时有，清风何处闻。相逢千里客，共醉百花春。小槛山当面，闲阶柳拂尘。何时卜西上，明月桂枝新。主[SEP][CLS]雨馀飞絮乱，相别思难任。酒罢河桥晚，帆开烟水深。蟾宫须展志，渔艇莫牵心。岐路从兹远，双鱼信勿沈。斋[SEP][CLS]偶自山僧院，移归傍砌栽。好风终日起，幽鸟有时来。筛月牵诗兴，笼烟伴酒杯。南窗睡轻起，萧飒雨声回。古[SEP][CLS]已是殊乡客，送君重惨然。河桥乍分首，槐柳正鸣蝉。短棹离幽浦，孤帆触远烟。清朝重文物，变化莫迁延。知[SEP][CLS]闲斋病初起，心绪复悠悠。开[UNK]群书蠹，听蝉满树秋。诗魔还渐动，药债未能酬。为忆前山色，扶持上小楼。主[SEP][CLS]送别人归春日斜，独鞭羸马指天涯。月生江上乡心动，投宿匆忙近酒家。[SEP][CLS]极目青青垄麦齐，野塘波阔下[UNK][UNK]。阳乌景暖林桑密，独立闲听戴胜啼。[SEP][CLS]位望谁能并，当年志已伸。人间传凤藻，天上演龙纶。贾马才无敌，褒雄誉益臻。除奸深系念，致主迥忘身。谏疏纵横上，危言果敢陈。忠贞虽贯世，消长岂由人。慷慨辞朝阙，迢遥涉路


In [556]:
model.eval()
text = '明月几时有，把酒问青天。不知'
indexed_token = tokenizer.encode(text)
for i in range(400 - len(text)):
    tokens_tensor = torch.tensor(indexed_token).to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)

    predictions = outputs[0]
    predicted_index = sample(predictions[-2, :].cpu().numpy())
    indexed_token[-1] = predicted_index
    indexed_token.append(102)
    if len(indexed_token) > 512:
        indexed_token = indexed_token[-512:]
print(tokenizer.decode(indexed_token))

[CLS] 明 月 几 时 有 ， 把 酒 问 青 天 。 不 知 何 处 醉 ， 定 与 故 人 同 。 主 [SEP] [CLS] 小 邑 沧 江 接 ， 新 亭 绿 柳 垂 。 晚 风 时 袅 袅 ， 柳 影 客 参 差 。 远 岫 何 当 有 ， 清 江 不 向 迟 。 因 高 聊 一 望 ， 非 是 酒 相 思 。 斋 [SEP] [CLS] 一 从 归 思 远 ， 长 忆 在 西 山 。 高 户 云 归 去 ， 人 家 月 上 还 。 无 才 堪 世 累 ， 多 病 见 人 闲 。 只 是 刘 桢 辈 ， 凌 云 未 拂 冠 。 古 [SEP] [CLS] 厌 见 移 家 远 ， 空 令 客 到 稀 。 故 园 花 未 发 ， 新 橘 酒 初 归 。 水 月 知 消 息 ， 山 寒 犹 未 归 。 可 怜 窗 下 月 ， 明 月 照 人 衣 。 知 [SEP] [CLS] 行 人 何 处 去 ， 浪 迹 洞 门 开 。 万 木 无 秋 色 ， 孤 帆 有 客 来 。 阴 阳 应 未 到 ， 天 末 又 衔 杯 。 去 去 相 思 否 ， 青 山 长 绿 苔 。 主 [SEP] [CLS] 霜 凋 古 木 边 ， 去 马 弟 兄 还 。 独 起 千 里 念 ， 空 为 千 里 还 。 海 阴 云 梦 泽 ， 山 色 雨 愁 山 。 明 发 新 知 少 ， 离 心 若 梦 间 。 斋 [SEP] [CLS] 林 端 花 覆 地 ， 夜 景 向 愁 生 。 数 雁 别 来 意 ， 片 帆 归 去 声 。 远 书 和 雁 响 ， 寒 渚 带 潮 声 。 况 是 东 归 客 ， 愁 人 正 字 行 。 古 [SEP] [CLS] 四 望 非 人 境 ， 应 无 乡 信 传 。 远 山 经 雨 后 ， 当 路 入 云 边 。 大 泽 云 鸿 去 ， 扁 舟 月 夜 圆 。 离 心 寄 西 北 ， 不 畏 海 棠 年 。 知 [SEP] [CLS] 万 里 杨 柳 色 ， 出 关 随 故 人 。 轻 烟 拂 地 [SEP]


In [558]:
model.eval()
text = '长铗归来兮，食无鱼。长铗归来兮，出无车。'
indexed_token = tokenizer.encode(text)
for i in range(400 - len(text)):
    tokens_tensor = torch.tensor(indexed_token).to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)

    predictions = outputs[0]
    predicted_index = sample(predictions[-2, :].cpu().numpy())
    indexed_token[-1] = predicted_index
    indexed_token.append(102)
    if len(indexed_token) > 512:
        indexed_token = indexed_token[-512:]
print(tokenizer.decode(indexed_token))

[CLS] 长 [UNK] 归 来 兮 ， 食 无 鱼 。 长 [UNK] 归 来 兮 ， 出 无 车 。 哀 哀 从 古 耶 ， 恻 怆 以 喜 言 。 家 既 名 从 ， 族 为 公 卿 。 将 眷 青 桑 兮 ， 心 希 遂 见 。 主 [SEP] [CLS] 海 水 无 梁 兮 ， 波 塞 抑 元 化 。 古 [SEP] [CLS] 山 海 超 忽 兮 ， 混 混 沌 相 向 。 洪 波 激 壮 浪 ， 怒 激 悲 风 雨 。 万 族 皆 茂 功 ， 播 于 多 年 少 。 斋 [SEP] [CLS] 奔 峭 迫 兮 ， 靡 迤 [UNK] 兮 。 水 攒 激 兮 ， 融 为 漪 涟 。 仙 菊 有 妍 兮 ， 盎 中 叔 之 。 祗 役 而 反 ， 尚 曰 云 之 人 。 古 [SEP] [CLS] 维 [UNK] 兮 ， 发 太 息 兮 ， 问 吴 之 人 。 山 海 凝 暮 兮 ， 怅 离 思 兮 。 谁 古 铸 镜 者 ， 犹 白 其 颦 。 知 [SEP] [CLS] 於 赫 兮 ， 尧 没 宗 兮 ， 皎 皎 兮 。 主 [SEP] [CLS] 沟 壑 之 人 兮 ， 播 桑 鸣 。 高 丘 之 人 兮 ， 于 山 于 门 兮 。 开 蒿 而 望 云 ， 于 上 君 兮 。 斋 [SEP] [CLS] 东 有 羲 和 兮 ， 靡 有 先 [UNK] 兮 ， 天 之 壤 兮 。 播 采 而 雨 ， 于 山 于 门 。 于 山 于 门 兮 ， 孰 知 其 然 。 周 粟 [UNK] 兮 ， 旨 酒 食 兮 。 村 有 子 兮 ， 我 有 臣 兮 。 古 [SEP] [CLS] （ 《 柳 》 ， 《 昭 夏 之 歌 》 ） 自 有 兴 王 ， 何 悟 其 荒 。 自 东 泛 舟 ， 乃 绵 邈 矣 。 周 力 衰 ， 王 道 无 归 。 我 祖 正 命 兮 ， 贻 谋 大 惠 。 （ 《 《 大 [UNK] 》 ） 。 斋 [SEP] [CLS] 有 熊 蹲 兮 ， 飞 次 [UNK] 兮 ， 翼 翼 翼 兮 ， 群 雌 兮 ， 吞 沧 仑 兮 。 吞 周 志 兮 ， 孰 [SEP]


In [565]:
model.eval()
text = '黄河之水天上来，奔流到海不复回。'
indexed_token = tokenizer.encode(text)
for i in range(400 - len(text)):
    tokens_tensor = torch.tensor(indexed_token).to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)

    predictions = outputs[0]
    predicted_index = sample(predictions[-2, :].cpu().numpy(), temperature=1.2)
    indexed_token[-1] = predicted_index
    indexed_token.append(102)
    if len(indexed_token) > 512:
        indexed_token = indexed_token[-512:]
print(tokenizer.decode(indexed_token))

[CLS] 黄 河 之 水 天 上 来 ， 奔 流 到 海 不 复 回 。 我 心 不 厌 碍 ， 因 欲 上 天 台 。 生 作 卧 龙 窟 ， 手 种 玉 山 禾 。 黄 鹤 有 二 毛 ， 蹭 蹬 为 龙 媒 。 今 朝 北 斗 向 北 落 ， 大 不 可 颓 ， 直 上 天 何 悠 哉 。 古 [SEP] [CLS] 昔 日 北 风 摧 ， 黄 金 四 散 灭 。 邑 中 有 老 男 ， 早 朝 常 对 阙 。 不 叹 妾 事 非 ， 自 叹 妾 缘 别 。 君 今 如 不 从 ， 倏 忽 复 如 何 。 斋 [SEP] [CLS] 紫 阁 连 天 都 ， 华 宫 阙 五 云 。 白 云 飘 怅 雪 ， 明 月 散 昆 峰 。 天 子 亦 薄 去 ， 孝 夫 亦 未 闻 。 行 云 飘 海 不 自 持 ， 望 水 杳 冥 何 岁 云 。 但 见 山 阴 人 ， 移 居 未 能 已 。 斋 [SEP] [CLS] 余 本 疏 放 士 ， [UNK] 来 非 傲 然 。 误 落 边 尘 中 ， 一 朝 风 景 鲜 。 朝 随 鸿 雁 去 ， 暮 逐 羔 雁 还 。 羞 作 边 庭 客 ， 悲 逢 落 叶 年 。 古 [SEP] [CLS] 余 本 疏 散 者 ， 偏 蒙 漂 泊 生 。 同 占 相 思 树 ， 不 叹 往 落 声 。 由 来 几 日 会 ， 每 见 海 潮 平 。 离 忧 如 边 草 ， 春 色 又 欲 生 。 湘 东 几 日 到 ， 攀 折 欲 垂 名 。 知 [SEP] [CLS] 种 田 东 山 下 ， 取 路 何 时 还 。 粟 苗 空 满 地 ， 年 年 桑 植 闲 。 垂 条 闲 古 墓 ， 是 处 即 乔 木 。 成 此 如 古 人 ， 登 临 叹 人 世 。 主 [SEP] [CLS] 种 田 东 山 下 ， 取 与 白 云 齐 。 树 色 不 隐 土 ， 水 声 无 落 鸡 。 涧 户 亦 有 草 ， 垄 头 多 是 溪 。 借 问 结 槃 客 ， [SEP]


In [583]:
model.eval()
text = '黄鹤一去不复返，白云千载空悠悠。'
indexed_token = tokenizer.encode(text)
for i in range(400 - len(text)):
    tokens_tensor = torch.tensor(indexed_token).to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)

    predictions = outputs[0]
    
    # Downgrade some options
    for _ in [102, 3153, 4761, 1367, 712, 100]:
        predictions[-2, _] -= 30
    
    predicted_index = sample(predictions[-2, :].cpu().numpy(), temperature=1.2)
    indexed_token[-1] = predicted_index
    indexed_token.append(102)
    if len(indexed_token) > 512:
        indexed_token = indexed_token[-512:]
print(tokenizer.decode(indexed_token))

[CLS] 黄 鹤 一 去 不 复 返 ， 白 云 千 载 空 悠 悠 。 浙 江 涛 惊 掠 水 魄 ， 杭 州 山 在 望 星 楼 。 楼 阁 乍 看 新 月 入 ， 风 帆 欲 过 重 江 头 。 山 城 七 泽 增 寒 色 ， 此 地 新 为 故 国 游 。 念 此 令 人 老 未 归 ， 且 凭 阑 干 斗 酒 歌 。 义 心 不 是 男 儿 事 ， 谩 向 诸 贤 名 利 机 。 请 君 且 看 洞 庭 石 ， 挂 席 停 舟 试 一 过 。 人 生 事 义 须 及 时 ， 莫 将 文 字 缚 风 波 。 兴 来 杰 气 若 可 夺 ， 双 鬓 终 垂 眼 前 丑 。 画 角 三 声 动 烟 水 ， 牧 童 胡 雏 鸢 起 饥 。 摇 鞭 骑 马 傍 寒 垄 ， 把 酒 相 看 泪 如 雨 。 长 笑 士 繇 最 苦 辛 ， 不 为 之 推 脉 与 尘 。 他 时 所 以 终 我 尔 ， 惠 然 而 我 忘 其 身 。 神 之 不 能 救 其 死 ， 亦 何 必 拟 将 救 君 。 劝 君 且 饮 酒 ， 酒 能 陶 令 巾 。 伊 昔 盛 才 豪 ， 激 士 如 纵 横 。 吞 声 不 许 义 ， 巧 佞 不 足 恃 。 所 以 尸 禄 役 ， 举 家 如 粪 土 。 金 盘 有 何 用 ， 玉 笼 无 由 指 。 生 君 父 母 年 ， 已 为 儒 翁 丑 。 我 贫 惠 君 子 ， 自 惭 名 与 利 。 所 以 尸 辱 间 ， 诚 不 敬 天 子 。 抚 心 欲 忍 言 ， 但 哭 苍 生 耳 。 儒 生 竟 如 何 ， 庸 蜀 功 成 矣 。 爱 惜 两 不 谐 ， 且 为 闲 人 致 。 市 井 徒 有 名 ， 泊 舟 今 已 矣 。 爱 君 似 我 心 ， 悲 喜 相 逢 喜 。 平 生 志 气 立 ， 义 合 从 如 此 。 所 以 沧 洲 方 ， 相 看 [SEP]


In [584]:
model.eval()
text = '黄河之水天上来，奔流到海不复回。'
indexed_token = tokenizer.encode(text)
for i in range(400 - len(text)):
    tokens_tensor = torch.tensor(indexed_token).to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)

    predictions = outputs[0]
    
    # Downgrade some options
    for _ in [102, 3153, 4761, 1367, 712, 100]:
        predictions[-2, _] -= 30
    
    predicted_index = sample(predictions[-2, :].cpu().numpy(), temperature=1.2)
    indexed_token[-1] = predicted_index
    indexed_token.append(102)
    if len(indexed_token) > 512:
        indexed_token = indexed_token[-512:]
print(tokenizer.decode(indexed_token))

[CLS] 黄 河 之 水 天 上 来 ， 奔 流 到 海 不 复 回 。 我 今 因 此 涕 涟 涟 ， 零 落 漂 母 忆 流 泉 。 昔 人 结 发 经 济 川 ， 今 人 结 发 竟 谁 在 。 幽 壤 不 曾 消 塞 色 ， 沧 海 有 还 君 旧 情 。 新 人 昔 人 爱 花 颜 ， 今 人 昔 人 共 几 全 。 且 与 坚 贞 同 一 心 ， 功 成 久 立 节 不 刊 。 况 复 秋 风 才 一 叹 ， 此 音 必 绝 岂 可 传 。 君 不 见 泰 山 高 枝 ， 春 风 为 我 结 其 根 。 黄 叶 从 风 散 ， 狂 花 随 风 翻 。 此 时 妾 弃 妾 ， 憔 悴 在 路 傍 。 不 惜 黄 金 买 ， 门 前 芳 草 生 。 感 君 下 山 去 ， 持 赠 此 床 人 。 忆 昔 公 家 闺 ， 玉 颜 艳 芳 菲 。 容 华 委 蔓 草 ， 一 去 无 还 期 。 桑 榆 日 已 颓 ， 哀 音 坐 相 思 。 待 君 幽 魂 意 ， 贞 叶 已 相 滋 。 无 用 还 对 此 ， 人 心 如 死 灰 。 寄 言 赤 玉 箫 ， 用 尽 双 飞 归 。 朱 颜 感 落 尽 ， 不 忍 生 黄 金 。 乡 国 有 昔 时 ， 深 宫 思 君 君 。 蔡 琰 薄 命 子 ， 竟 为 云 雨 灾 。 大 道 今 不 嗣 ， 贱 妾 将 安 辞 。 斫 石 作 高 火 ， 疏 凿 捐 清 源 。 炼 玉 且 不 用 ， 况 乃 空 为 雌 。 金 膏 倘 可 尽 ， 他 人 以 为 谁 。 悠 悠 浮 云 水 ， 一 濯 无 近 圆 。 君 看 鸿 雁 来 ， 哀 鸣 何 由 缘 。 彩 曲 不 可 说 ， 使 我 肠 断 绝 。 愿 持 一 书 卷 ， 以 代 双 玉 盘 。 流 泉 倘 相 从 ， 灭 石 为 君 弹 。 幽 闺 一 夕 梦 ， 幽 梦 [SEP]


In [592]:
model.eval()
text = '君不见黄河之水天上来，奔流到海不复回。君不见高堂明镜悲白发，朝如青丝暮成雪。'
indexed_token = tokenizer.encode(text)
for i in range(500 - len(text)):
    tokens_tensor = torch.tensor(indexed_token).to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)

    predictions = outputs[0]
    
    # Downgrade some options
    for _ in [102, 3153, 4761, 1367, 712, 100]:
        predictions[-2, _] -= 30
    
    predicted_index = sample(predictions[-2, :].cpu().numpy(), temperature=1.3)
    indexed_token[-1] = predicted_index
    indexed_token.append(102)
    if len(indexed_token) > 512:
        indexed_token = indexed_token[-512:]
print(tokenizer.decode(indexed_token))

[CLS] 君 不 见 黄 河 之 水 天 上 来 ， 奔 流 到 海 不 复 回 。 君 不 见 高 堂 明 镜 悲 白 发 ， 朝 如 青 丝 暮 成 雪 。 人 生 得 意 须 尽 欢 ， 莫 使 金 樽 空 对 月 。 天 生 我 材 必 有 用 ， 千 金 散 尽 还 复 来 。 烹 羊 宰 牛 且 为 乐 ， 会 须 一 饮 三 百 杯 。 岑 夫 子 ， 丹 丘 生 ， 将 进 酒 ， 君 莫 停 。 与 君 歌 一 曲 ， 请 君 为 我 侧 耳 听 。 钟 鼓 馔 玉 不 足 贵 ， 但 愿 长 醉 不 愿 醒 。 悬 河 捧 日 酒 ， 为 君 击 壤 人 。 风 吹 棠 梨 花 ， 此 时 陋 巷 老 。 鲁 酒 哂 ， 为 君 歌 一 声 。 钟 陵 虽 有 酒 ， 与 君 烂 漫 争 天 倾 。 富 贵 无 限 杯 ， 贤 愚 不 重 陈 。 请 君 金 一 斗 ， 酒 中 与 我 醉 。 歌 今 虽 有 言 ， 劝 君 为 我 倾 。 我 醉 向 阳 台 ， 梦 中 归 取 醉 。 愿 君 醉 似 月 ， 流 影 到 君 前 。 照 水 玉 堂 开 ， 当 炉 美 人 连 。 愿 君 弹 一 曲 ， 直 到 太 平 年 。 昔 闻 草 圣 人 ， 今 听 马 相 牵 。 安 得 不 尽 意 ， 始 从 肘 后 开 。 舒 卷 随 九 风 ， 若 流 涕 泗 下 。 举 酒 劝 行 乐 ， 坟 前 不 见 山 。 路 旁 有 寒 月 ， 千 里 鸣 刀 钱 。 去 去 来 几 时 ， 惆 怅 清 洛 间 。 朱 颜 反 不 顾 ， 陋 巷 空 荒 山 。 君 不 见 青 云 之 游 宦 人 ， 秉 心 为 君 颜 。 有 时 结 心 事 ， 倏 忽 歧 路 斑 。 高 堂 列 红 烛 ， 四 望 高 且 闲 。 东 门 有 光 华 ， 开 帘 揽 群 山 。 但 恐 岁 月 晚 ， 坐 客 岩 之 间 。 高 堂 列 红 烛 ， 坐 客 弹 素 琴 。 此 意 竟 不 足 ， 白 云 空 自 闲 。 况 当 今 夜 别 ， 长 欲 片 时 还 。 持 杯 对 新 酒 ， 勿 结 同 心 颜 。 此 会 难 再 得 ， 酣 歌 且 裴 回 。 人 生 百 年 中 ， 有 酒 胜 余 颜 。 且 须 劝 加 我 ， 少 欢 即 我 闲 。 相 

In [588]:
model.eval()
text = '山有木兮木有枝，心悦君兮君不知。'
indexed_token = tokenizer.encode(text)
for i in range(400 - len(text)):
    tokens_tensor = torch.tensor(indexed_token).to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)

    predictions = outputs[0]
    
    # Downgrade some options
    for _ in [102, 3153, 4761, 1367, 712, 100]:
        predictions[-2, _] -= 30
    
    predicted_index = sample(predictions[-2, :].cpu().numpy(), temperature=1.2)
    indexed_token[-1] = predicted_index
    indexed_token.append(102)
    if len(indexed_token) > 512:
        indexed_token = indexed_token[-512:]
print(tokenizer.decode(indexed_token))

[CLS] 山 有 木 兮 木 有 枝 ， 心 悦 君 兮 君 不 知 。 当 无 有 用 兮 不 倚 ， 蹇 独 好 之 容 易 为 。 感 人 兮 不 顾 ， 眷 言 兮 不 怡 。 月 皎 日 兮 徒 光 ， 星 离 雨 兮 不 休 。 耿 耿 日 兮 不 寐 ， 嬉 游 兮 不 归 。 余 莫 厌 糠 核 兮 ， 又 奚 斯 失 兮 。 方 三 日 兮 不 分 ， 澹 长 路 兮 摧 颓 。 失 天 地 兮 不 我 ， 蹇 独 何 之 摧 。 生 万 化 兮 汝 能 全 ， 生 不 达 兮 自 然 。 欲 赠 言 兮 空 踟 躇 ， 问 其 人 兮 不 闻 。 昔 在 山 兮 今 在 山 ， 山 之 幽 兮 不 见 。 时 出 谷 兮 草 木 ， 日 入 城 兮 车 马 。 周 回 兮 不 见 ， 远 望 兮 空 见 。 寄 书 兮 欲 寄 书 ， 见 君 兮 欲 收 。 日 长 路 兮 马 首 ， 夜 长 桥 兮 空 还 。 远 见 青 山 兮 不 见 ， 心 思 隐 兮 湖 水 空 。 日 暮 兮 将 短 ， 人 归 山 兮 夕 烟 。 （ 观 花 萼 楼 前 ， 《 纪 事 》 ） 。 密 雨 霏 霏 兮 自 萧 索 ， 不 觉 委 颜 兮 复 几 。 （ 为 长 安 兮 不 见 ， 即 安 可 以 登 高 台 。 子 之 歌 兮 ： 将 太 公 兮 ， 日 隐 暮 兮 不 来 ， 登 云 台 兮 不 来 。 此 兮 欲 征 山 ， 望 青 山 兮 不 从 。 山 有 桂 兮 欲 凋 ， 白 云 兮 欲 没 。 洪 亭 一 望 兮 ， 山 有 石 兮 水 无 津 。 我 独 夫 兮 欲 征 山 ， 君 独 往 兮 欲 征 还 。 梁 有 石 兮 颍 水 ， 思 松 老 兮 欲 征 还 。 曾 未 还 兮 颍 水 上 ， 日 摇 洲 兮 空 叹 息 。 秦 人 兮 欲 泛 [SEP]


In [589]:
model.eval()
text = '若有人兮天山之间，被薜荔兮带女萝。'
indexed_token = tokenizer.encode(text)
for i in range(400 - len(text)):
    tokens_tensor = torch.tensor(indexed_token).to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)

    predictions = outputs[0]
    
    # Downgrade some options
    for _ in [102, 3153, 4761, 1367, 712, 100]:
        predictions[-2, _] -= 30
    
    predicted_index = sample(predictions[-2, :].cpu().numpy(), temperature=1.2)
    indexed_token[-1] = predicted_index
    indexed_token.append(102)
    if len(indexed_token) > 512:
        indexed_token = indexed_token[-512:]
print(tokenizer.decode(indexed_token))

[CLS] 若 有 人 兮 天 山 之 间 ， 被 [UNK] 荔 兮 带 女 萝 。 感 深 柔 兮 情 思 深 ， 赠 远 人 兮 欲 征 心 。 朝 驰 夫 兮 夕 为 。 延 年 妹 兮 日 将 暮 ， 暮 入 兮 日 初 长 。 万 里 兮 不 留 ， 千 里 兮 不 留 。 黄 鹤 兮 归 来 ， 应 青 山 兮 怨 思 悠 悠 。 澧 之 曲 兮 ， 君 归 去 兮 夕 阳 。 橹 丝 芳 芷 兮 ， 荃 壁 朱 兮 ， 终 不 见 于 斯 。 欲 浮 云 兮 不 从 ， 山 之 幽 兮 雨 兮 夕 阳 。 晚 阴 阴 兮 幂 幂 ， 水 连 天 兮 夕 阳 。 白 云 自 从 兮 马 群 ， 爱 此 山 之 白 云 。 欲 往 从 兮 青 山 里 ， 思 归 兮 夕 阳 。 （ 苦 雨 ， 见 此 山 之 白 云 ） 。 掩 石 兮 紫 烟 ， 水 连 天 兮 不 闻 。 与 我 兮 欲 相 依 ， 恐 碧 草 兮 令 人 幽 。 （ 采 瑶 琴 》 ） 。 取 金 山 之 白 兮 ， 将 乘 飞 雨 ， 采 清 潭 之 绿 芷 。 ） 。 笑 看 垢 尘 之 忽 忆 ， 却 笑 （ 谬 席 为 郎 ， 见 《 语 林 》 ） 。 不 能 别 ， 愿 见 朗 月 之 明 。 （ 幽 岩 之 列 ， 惧 淹 留 ， 不 得 见 东 西 南 北 三 千 里 ， 卧 松 之 白 云 。 ） 。 非 关 塞 之 咽 ， 势 不 见 之 者 默 矣 。 ） 。 早 晚 兮 乘 白 云 ， 复 闻 若 有 情 。 见 此 山 之 秀 兮 ， 顿 然 若 有 意 兮 。 松 石 顾 盼 ， 如 何 兮 。 群 山 长 兮 ， 猿 哀 鸣 兮 ， 诵 霜 之 声 。 客 自 经 此 兮 ， 于 山 之 上 兮 。 却 忆 山 之 幽 幽 兮 ， 便 辞 家 兮 不 来 。 爱 枫 林 之 白 [SEP]


In [590]:
model.eval()
text = '离离原上草，一岁一枯荣。'
indexed_token = tokenizer.encode(text)
for i in range(400 - len(text)):
    tokens_tensor = torch.tensor(indexed_token).to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)

    predictions = outputs[0]
    
    # Downgrade some options
    for _ in [102, 3153, 4761, 1367, 712, 100]:
        predictions[-2, _] -= 30
    
    predicted_index = sample(predictions[-2, :].cpu().numpy(), temperature=1.2)
    indexed_token[-1] = predicted_index
    indexed_token.append(102)
    if len(indexed_token) > 512:
        indexed_token = indexed_token[-512:]
print(tokenizer.decode(indexed_token))

[CLS] 离 离 原 上 草 ， 一 岁 一 枯 荣 。 野 水 冰 难 释 ， 空 郊 云 乍 生 。 唯 应 北 窗 下 ， 明 日 为 谁 倾 。 心 极 在 飞 鸟 ， 不 劳 伤 别 情 。 兹 年 如 昨 日 ， 微 雨 自 为 情 。 海 上 一 相 见 ， 高 阳 初 至 明 。 此 时 春 已 晚 ， 此 日 泪 盈 盈 。 忆 昔 霍 家 公 ， 种 田 长 松 柏 。 萧 条 属 秋 节 ， 寂 寞 临 风 景 。 灼 灼 园 中 树 ， 蔼 蔼 桑 榆 荣 。 秋 山 有 露 气 ， 此 日 无 人 耕 。 我 来 抚 棋 观 ， 忽 若 怀 其 精 。 慷 慨 吐 歌 笑 ， 谁 能 识 其 诚 。 留 连 树 栖 鸟 ， 千 载 各 何 情 。 夫 子 门 既 毁 ， 世 人 心 自 惊 。 此 人 犹 未 死 ， 所 以 慰 吾 诚 。 此 客 共 悬 圃 ， 相 对 时 一 倾 。 君 子 山 岳 期 ， 我 心 安 可 平 。 谁 谓 尔 无 责 ， 我 心 同 所 营 。 相 去 复 相 别 ， 重 来 复 一 行 。 他 辰 且 携 手 ， 欢 乐 在 此 生 。 倘 能 握 手 足 ， 何 用 念 形 形 神 明 。 相 与 会 合 天 及 我 ， 更 问 此 人 情 。 一 别 苟 既 别 ， 寸 心 宁 自 平 。 子 在 淮 南 地 ， 我 归 洛 阳 城 。 秋 深 夜 风 起 ， 露 下 河 汉 明 。 复 有 离 别 处 ， 还 为 千 里 行 。 河 边 杨 柳 枝 ， 相 见 几 回 惊 。 相 忆 不 可 见 ， 相 思 表 道 情 。 无 期 一 杯 酒 ， 可 以 慰 心 情 。 相 见 不 相 识 ， 相 思 复 相 迎 。 何 时 共 携 手 ， 与 我 此 三 征 。 欢 乐 与 欢 娱 ， 混 然 万 里 情 。 [SEP]


In [593]:
model.eval()
text = '人生若只如初见，'
indexed_token = tokenizer.encode(text)
for i in range(500 - len(text)):
    tokens_tensor = torch.tensor(indexed_token).to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)

    predictions = outputs[0]
    
    # Downgrade some options
    for _ in [102, 3153, 4761, 1367, 712, 100]:
        predictions[-2, _] -= 30
    
    predicted_index = sample(predictions[-2, :].cpu().numpy(), temperature=1.3)
    indexed_token[-1] = predicted_index
    indexed_token.append(102)
    if len(indexed_token) > 512:
        indexed_token = indexed_token[-512:]
print(tokenizer.decode(indexed_token))

[CLS] 人 生 若 只 如 初 见 ， 何 事 先 归 不 见 春 。 君 近 东 西 两 相 见 ， 为 言 举 酒 任 所 亲 。 且 倾 一 尊 与 我 饮 ， 何 必 多 情 相 劝 人 。 桃 李 栽 来 几 度 春 ， 似 我 今 朝 似 去 人 。 色 荒 碧 树 花 尽 老 ， 醉 乡 潜 送 一 车 轮 。 归 去 来 兮 忆 此 身 ， 不 属 花 时 独 醉 春 。 今 日 不 忘 君 自 醉 ， 醉 年 无 伴 醉 年 人 。 长 安 千 里 万 里 别 ， 明 月 几 回 来 照 秦 。 渭 水 桥 南 音 信 绝 ， 关 山 路 上 权 关 绝 。 竞 把 酒 杯 来 相 问 ， 不 离 别 后 能 伤 别 。 遥 见 明 星 照 天 末 ， 秋 风 萧 索 两 河 清 。 明 月 已 殁 子 规 啼 ， 浮 云 可 望 不 可 道 。 美 人 兮 夜 向 东 来 ， 明 月 照 心 堪 射 杀 。 怜 复 畏 落 日 已 斜 ， 恩 不 断 ， 一 回 断 绝 不 可 裁 。 相 劝 一 行 泪 如 雨 ， 愿 书 千 断 肠 。 黄 河 捧 天 子 ， 大 车 囊 括 绝 。 四 海 有 梦 之 路 无 ， 千 龄 无 路 亦 不 见 。 夜 分 半 ， 梦 中 何 处 令 人 传 。 长 安 路 ， 月 照 霜 树 枝 ， 南 枝 家 住 时 。 仰 面 诉 天 子 ， 亲 劳 问 织 丝 。 行 人 不 可 见 ， 暗 哭 苍 门 碑 。 彼 见 中 尚 羞 ， 白 日 为 尘 滋 。 昔 年 在 东 郡 ， 今 日 成 老 时 。 容 颜 不 可 说 ， 赫 赫 宁 相 欺 。 生 者 能 几 何 ， 短 歌 聊 自 持 。 耽 酒 持 一 杯 ， 有 愧 非 所 悲 。 男 儿 行 自 楚 ， 腹 病 不 得 宜 。 自 陈 一 盂 酒 ， 生 计 只 有 遗 。 奈 何 有 织 者 ， 不 奈 长 夜 时 。 啼 到 晓 夫 店 ， 形 影 心 相 随 。 昔 怜 李 侍 郎 ， 养 得 鸳 鸯 儿 。 今 日 在 长 安 ， 自 照 渭 城 陲 。 御 史 忽 然 死 ， 贱 妾 誓 不 遗 。 伤 心 与 夫 妇 ， 化 作 黄 凤 诗 。 吟 罢 两 惆 怅 ， 七 十 有 八 期 。 今 日 阳 春 花 ， 又 

In [596]:
model.eval()
text = '仿佛兮若轻云之闭月，'
indexed_token = tokenizer.encode(text)
for i in range(500 - len(text)):
    tokens_tensor = torch.tensor(indexed_token).to('cuda')
    with torch.no_grad():
        outputs = model(tokens_tensor)

    predictions = outputs[0]
    
    # Downgrade some options
    for _ in [102, 3153, 4761, 1367, 712, 100]:
        predictions[-2, _] -= 30
    
    predicted_index = sample(predictions[-2, :].cpu().numpy(), temperature=1.3)
    indexed_token[-1] = predicted_index
    indexed_token.append(102)
    if len(indexed_token) > 512:
        indexed_token = indexed_token[-512:]
print(tokenizer.decode(indexed_token))

[CLS] 仿 佛 兮 若 轻 云 之 闭 月 ， 耿 耿 不 逾 千 。 心 若 玉 兮 流 泉 兮 流 泉 ， 中 如 心 兮 列 群 英 。 千 年 万 岁 兮 涵 炯 ， 群 芳 歇 兮 流 水 。 使 我 心 兮 苦 既 异 ， 伤 不 已 而 今 。 愿 荷 君 兮 暂 辍 ， 长 风 愁 思 再 理 兮 。 愁 桂 影 兮 杨 柳 ， 横 蛾 眉 兮 越 王 台 。 蟠 金 石 兮 草 木 春 ， 山 山 魅 兮 愁 氛 氲 。 山 疑 黛 兮 水 蹇 ， 水 如 牙 兮 松 如 雪 。 蕙 为 襟 兮 烟 幂 幂 ， 伤 不 加 兮 空 自 伤 。 思 千 里 兮 涵 碧 空 ， 仙 路 远 兮 怨 空 寻 。 瑶 琴 弹 瑟 兮 已 盈 ， 心 亦 绝 兮 意 难 穷 。 思 一 见 兮 夺 晚 妆 ， 重 华 窗 兮 月 苍 苍 。 夜 既 几 兮 忆 群 玉 ， 君 岂 乏 兮 意 久 长 。 人 悄 悄 兮 夜 方 半 ， 孤 灯 孤 兮 愁 空 堂 。 锦 衾 冷 ， 香 亦 销 ， 夜 魂 悠 悠 。 客 有 梦 兮 意 未 已 ， 夜 未 央 兮 空 自 伤 。 闻 昔 人 兮 将 白 日 ， 魂 已 归 兮 将 何 方 。 空 房 寂 寂 兮 无 事 ， 落 花 纷 纷 兮 欲 黄 。 空 房 寂 寂 兮 有 人 ， 宿 在 月 兮 泪 相 濡 。 春 日 迟 ， 伤 不 得 兮 不 自 持 。 对 酒 时 容 兮 欲 谁 ， 忆 长 时 兮 已 焉 。 一 见 一 笑 兮 复 一 欢 ， 使 我 再 理 兮 欲 何 为 。 愿 再 拜 兮 欲 跪 ， 愿 再 拜 兮 何 时 。 愿 再 拜 兮 欲 死 ， 愿 将 愿 兮 欲 死 。 愿 天 子 兮 请 天 子 ， 无 令 然 后 ， 长 信 在 我 兮 久 留 。 久 不 见 兮 日 迟 ， 穆 王 公 兮 从 北 来 。 久 不 见 兮 龙 升 ， 重 就 掌 兮 时 物 添 。 常 恐 此 兮 来 归 兮 不 归 ， 幸 先 歇 兮 天 地 。 老 偏 貌 兮 好 颜 ， 君 岂 不 见 兮 马 如 箭 飞 。 且 须 一 饮 ， 往 往 之 箕 稀 。 已 耶 ， 没 胡 之 中 。 近 秦 之 西 ， 隔 河 兮 不 闻 语 。 本 不 见 乎 ， 念 我 与 尔 辞 ？ 

# Session Info

In [1]:
! pip freeze

absl-py==0.7.0
adjustText==0.7.3
aiohttp==3.8.1
aiosignal==1.2.0
altair==3.2.0
anndata==0.7.5
argh==0.26.2
ase==3.21.1
astor==0.7.1
astropy==3.2.1
async-timeout==4.0.2
asynctest==0.13.0
atomicwrites==1.3.0
attrs==19.1.0
autograd==1.3
autograd-gamma==0.5.0
backcall==0.1.0
base58==1.0.3
bleach==1.5.0
blinker==1.4
boto3==1.10.15
botocore==1.13.15
Bottleneck==1.2.1
cachetools==4.1.0
certifi==2021.5.30
chardet==3.0.4
charset-normalizer==2.0.4
Click==7.0
cvxopt==1.2.3
cvxpy==1.0.24
cycler==0.10.0
Cython==0.29.15
dataclasses==0.8
datasets==1.17.0
decorator==4.4.0
defusedxml==0.5.0
dill==0.2.9
docutils==0.15.2
ecos==2.0.7.post1
entrypoints==0.3
enum-compat==0.0.3
environment-kernels==1.1.1
fastcluster==1.2.4
filelock==3.4.1
FITS-tools==0.2
formulaic==0.2.4
frozenlist==1.2.0
fsspec==2022.1.0
future==0.17.1
gast==0.2.2
get-version==2.1
google-auth==1.14.3
google-auth-oauthlib==0.4.1
google-pasta==0.2.0
googledrivedownloader==0.4
grpcio==1.29.0
h5py==2.10.0
html5lib==0.9999999
huggingface-hub==0.